<a href="https://colab.research.google.com/github/jacquesbilombe/Plustrategy/blob/main/datafetcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For more information on the API, check this [link](https://pypi.org/project/sportradar/).

In [1]:
import os
import sys
import datetime
import requests
import pandas as pd

In [2]:
'''
url = "https://api.sportradar.com/soccer/trial/v4/en/competitions.json?api_key=98TmLtpZ0c85NST01jrvB5LoMI9mSgKD8Oh3tbz3"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)
# Parse the JSON response
data = response.json()

# Extract the relevant data from the JSON
competitions = data['competitions']
competition_names = [competition['name'] for competition in competitions]
competition_ids = [competition['id'] for competition in competitions]

# Create a DataFrame
df = pd.DataFrame({'Name': competition_names, 'ID': competition_ids})

# Print the DataFrame
print(df.to_string())'''


'\nurl = "https://api.sportradar.com/soccer/trial/v4/en/competitions.json?api_key=98TmLtpZ0c85NST01jrvB5LoMI9mSgKD8Oh3tbz3"\n\nheaders = {"accept": "application/json"}\n\nresponse = requests.get(url, headers=headers)\n# Parse the JSON response\ndata = response.json()\n\n# Extract the relevant data from the JSON\ncompetitions = data[\'competitions\']\ncompetition_names = [competition[\'name\'] for competition in competitions]\ncompetition_ids = [competition[\'id\'] for competition in competitions]\n\n# Create a DataFrame\ndf = pd.DataFrame({\'Name\': competition_names, \'ID\': competition_ids})\n\n# Print the DataFrame\nprint(df.to_string())'

In [15]:
url = "https://api.sportradar.com/soccer-extended/production/v4/en/schedules/2024-04-30/summaries.json?api_key=98TmLtpZ0c85NST01jrvB5LoMI9mSgKD8Oh3tbz3"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)


In [16]:
# prompt: put response into a dataframe

# Parse the JSON response
data = response.json()

# Extract the relevant data from the JSON
summaries = data['summaries']


In [17]:
# Initialize an empty list to hold extracted data
data_list = []

# Loop through each summary and extract information
for data in summaries:
    country = data['sport_event']['sport_event_context']['category']['name']
    sport_event_context = data['sport_event']['sport_event_context']
    competition = sport_event_context['competition']
    competitors = data['sport_event']['competitors']

    sport_name = sport_event_context['sport']['name']
    competition_name = competition['name']
    competition_gender = competition['gender']
    competitors_names = ', '.join([competitor['name'] for competitor in competitors])

    # Append extracted information to the list
    data_list.append({'Competition Name': competition_name, 'Location':country, 'Gender': competition_gender, 'Teams': competitors_names})

# Create DataFrame from the list
df = pd.DataFrame(data_list)

In [18]:
df.head(15)

,Competition Name,Location,Gender,Teams
0,LigaPro Primera A,Ecuador,men,"CD El Nacional, Delfin SC"
1,Primera Division,Chile,men,"Audax Italiano, Cobresal"
2,Primera B,Chile,men,"Deportes Limache, Deportes La Serena"
3,"Liga MX, Women, Clausura",Mexico,women,"CF Tigres UANL, Club Necaxa"
4,Liga 1,Peru,men,"CD Universidad Cesar Vallejo, Sporting Cristal"
5,Premier League,Jamaica,men,"Tivoli Gardens, Waterhouse FC"
6,"Liga de Ascenso, Clausura",Costa Rica,men,"AD Municipal Santa Ana, CD Escorpiones Belen"
7,"Liga MX, Women, Clausura",Mexico,women,"Club Leon, Club Santos Laguna"
8,"U20 CONMEBOL, Women",International Youth,women,"Argentina, Paraguay"
9,"Liga MX, Women, Clausura",Mexico,women,"FC Juarez, CF Monterrey"


In [26]:
import requests

# Set up the request headers with your API token
headers = {
    'X-Auth-Token': '62da988175c44392a3360f0dfcacef88'
}

# Make the GET request
response = requests.get('https://api.football-data.org/v4/teams/86/matches?status=SCHEDULED', headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Extract matches data
    matches = data['matches']

    # Create DataFrame
    df = pd.DataFrame(matches)
else:
    # Print an error message if the request was unsuccessful
    print(f"Error: {response.status_code}")


In [27]:
df

,area,competition,season,id,utcDate,status,matchday,stage,group,lastUpdated,homeTeam,awayTeam,score,odds,referees
0,"{'id': 2224, 'name': 'Spain', 'code': 'ESP', '...","{'id': 2014, 'name': 'Primera Division', 'code...","{'id': 1577, 'startDate': '2023-08-13', 'endDa...",438812,2024-05-04T14:15:00Z,TIMED,34,REGULAR_SEASON,None,2024-05-02T15:21:37Z,"{'id': 86, 'name': 'Real Madrid CF', 'shortNam...","{'id': 264, 'name': 'Cádiz CF', 'shortName': '...","{'winner': None, 'duration': 'REGULAR', 'fullT...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
1,"{'id': 2077, 'name': 'Europe', 'code': 'EUR', ...","{'id': 2001, 'name': 'UEFA Champions League', ...","{'id': 1630, 'startDate': '2023-09-19', 'endDa...",451684,2024-05-08T19:00:00Z,TIMED,2,SEMI_FINALS,None,2024-05-02T15:21:37Z,"{'id': 86, 'name': 'Real Madrid CF', 'shortNam...","{'id': 5, 'name': 'FC Bayern München', 'shortN...","{'winner': None, 'duration': 'REGULAR', 'fullT...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
2,"{'id': 2224, 'name': 'Spain', 'code': 'ESP', '...","{'id': 2014, 'name': 'Primera Division', 'code...","{'id': 1577, 'startDate': '2023-08-13', 'endDa...",438820,2024-05-11T16:30:00Z,TIMED,35,REGULAR_SEASON,None,2024-05-02T15:21:37Z,"{'id': 83, 'name': 'Granada CF', 'shortName': ...","{'id': 86, 'name': 'Real Madrid CF', 'shortNam...","{'winner': None, 'duration': 'REGULAR', 'fullT...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
3,"{'id': 2224, 'name': 'Spain', 'code': 'ESP', '...","{'id': 2014, 'name': 'Primera Division', 'code...","{'id': 1577, 'startDate': '2023-08-13', 'endDa...",438830,2024-05-14T19:30:00Z,TIMED,36,REGULAR_SEASON,None,2024-05-02T15:21:37Z,"{'id': 86, 'name': 'Real Madrid CF', 'shortNam...","{'id': 263, 'name': 'Deportivo Alavés', 'short...","{'winner': None, 'duration': 'REGULAR', 'fullT...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
4,"{'id': 2224, 'name': 'Spain', 'code': 'ESP', '...","{'id': 2014, 'name': 'Primera Division', 'code...","{'id': 1577, 'startDate': '2023-08-13', 'endDa...",438842,2024-05-19T00:00:00Z,SCHEDULED,37,REGULAR_SEASON,None,2024-05-02T15:21:37Z,"{'id': 94, 'name': 'Villarreal CF', 'shortName...","{'id': 86, 'name': 'Real Madrid CF', 'shortNam...","{'winner': None, 'duration': 'REGULAR', 'fullT...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
5,"{'id': 2224, 'name': 'Spain', 'code': 'ESP', '...","{'id': 2014, 'name': 'Primera Division', 'code...","{'id': 1577, 'startDate': '2023-08-13', 'endDa...",438850,2024-05-26T00:00:00Z,SCHEDULED,38,REGULAR_SEASON,None,2024-05-02T15:21:37Z,"{'id': 86, 'name': 'Real Madrid CF', 'shortNam...","{'id': 90, 'name': 'Real Betis Balompié', 'sho...","{'winner': None, 'duration': 'REGULAR', 'fullT...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
